In [9]:
# !pip install bs4
# !pip install webdriver_manager
# !pip install selenium==3.141.0

### 네이버 카페 크롤러

In [10]:
# selenium 셋팅
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [11]:
import time
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup as bs

In [12]:
from selenium.webdriver.common.by import By

### 네이버 로그인
자동화된 크롬창을 실행시켜서 네이버 로그인 페이지 접속
1. 크롬드라이버 제어(크롬창 켜기)
2. 네이버 로그인(아이디 입력 -> 비밀번호 입력 -> 로그인 버튼 클릭 순서로 작업 진행)

In [13]:
# 크롬 드라이버 제어 
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')

#현재 컴퓨터 크롬 드라이버 위치
chrome_path = r"C:\Users\user\chromedriver.exe"

#자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path = chrome_path, options = chrome_options)

In [14]:
# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

# 본인의 아이디, 비밀번호를 각 변수에 저장
my_id = 'heosb1'
my_pw = 'tjdqordl0108!'

# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element('id','log.login').click()
time.sleep(1)

In [15]:
# 카페 접속
time.sleep(1)
doctors_url = 'https://cafe.naver.com/2myloft'
driver.get(doctors_url)
time.sleep(1)

# 원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink34"]').click()
time.sleep(1)

# 프레임 전환
driver.switch_to.frame('cafe_main')
time.sleep(1)

### 게시물 크롤링
1. 첫번째 게시물 클릭 -XPath 활용
2. 크롤링할 개수 지정
3. 크롤링

In [16]:
# 제목, 본문, 댓글, 시간 빈 list 생성\
titles = []
articles = []
comments = []
dates = []

count_view = 0
# 1. 첫 번째 게시물 클릭 (선택자 입력 : XPath)
driver.find_element('xpath', '//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()

# 2. 크롤링(for문으로 지정한 게시물 개수까지 돌림)
for i in range(97): #새로 창 열기
    for j in range(50): # 한 번에 몇 개의 게시물을 크롤링할 것인가
        driver.implicitly_wait(30)
        # 제목 수집
        time.sleep(random.uniform(0.5,1))
        title = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div').text
        titles.append(title)
        # 본문 수집 (3가지 경우 존재)
        try:
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text').text
            articles.append(content)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR, 'div.ContentRenderer').text
                articles.append(content)
            except:
                    content = [] # 본문이 사진으로 된 경우
                    articles.append(content)
        # 댓글 수집 (댓글이 없는 경우, 댓글이 있는 경우 2가지 존재)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지고 오기
        iscomment = soup.find_all('span', class_='text_comment')
        if len(iscomment) == 0: # 댓글 없는 경우
            box = []
        else: # 댓글 있는 경우
            WebDriverWait(driver,15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')))
            # 웹 드라이버를 통해 브라우저에서 최대 15초 기다려주고, text_comment(댓글 선택자)가 나올 대까지...
            soup = bs(driver.page_source,'lxml')
            iscomment = soup.find_all('span', class_='text_comment')
            box = []
            for i in iscomment:
                box.append([i.get_text()])
        
        comments.append(box)
            
        # 날짜 수집
        time.sleep(random.uniform(0.5,1))
        date = driver.find_element('xpath', '//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        dates.append(date)
        
        # count_view 크롤링 진행 상황(50개마다 출력)
        count_view += 1
        if count_view%50 == 0: # 50개마다 진행 상황 출력
            print('{}개 게시물 크롤링 완료'.format(count_view))
        else:
            pass
        
        try:
            driver.find_element(By.CSS_SELECTOR, "#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default > span").click()
            driver.implicitly_wait(28)
        except:
            current_url = driver.current_url
            driver.close()
            
            # 크롬창 다시 열기
            driver = webdriver.Chrome(executable_path = chrome_path, options = chrome_options)

            # 재로그인
            # 네이버 화면 이동
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(10)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)

            # '로그인' 버튼 클릭
            driver.find_element('id','log.login').click()
            time.sleep(1)

            # 멈춘 링크부터 가져오기
            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)
    
    # 크롬이 다시 열릴 때 가지고 올 다음글 링크
    link = driver.find_element('xpath','//*[@id="app"]/div/div/div[1]/div[2]/a[2]').get_attribute('href')

    # 현재 크롬창 닫기
    driver.close()

    # 다시 열기
    driver = webdriver.Chrome(executable_path = chrome_path, options = chrome_options)

    # 재로그인
    # 네이버 화면 이동
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(10)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    # '로그인' 버튼 클릭
    driver.find_element('id','log.login').click()
    time.sleep(1)
    
    # 다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')

#크롬창 닫기
driver.quit()

50개 게시물 크롤링 완료
100개 게시물 크롤링 완료
150개 게시물 크롤링 완료
200개 게시물 크롤링 완료


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=107.0.5304.107)


In [ ]:
titles

In [ ]:
articles

In [ ]:
comments

In [ ]:
dates

In [ ]:
print(len(titles))
print(len(articles))
print(len(comments))
print(len(dates))

In [ ]:
data = {'time' : dates, 'title' : titles, 'article' : articles, 'comment' : comments}

dataDF = pd.DataFrame(data)

In [ ]:
dataDF

In [ ]:
dataDF.to_csv("피터팬.csv", encoding = "UTF-8-sig")